In [2]:
#
# grab some dependencies for our project
#
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime as time
from forex_python.converter import CurrencyRates
import requests

start = time.now()

In [3]:
#
# pull in the list of transactions
#
df = pd.read_csv('transactions.csv')

In [4]:
#
# take a look at what we're working with
#
print(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69469 entries, 0 to 69468
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     69469 non-null  int64  
 1   customer_id            69469 non-null  int64  
 2   customer_country_code  69469 non-null  object 
 3   processed_at           69469 non-null  object 
 4   currency               69469 non-null  object 
 5   amount                 55576 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 3.2+ MB
None


,id,customer_id,customer_country_code,processed_at,currency,amount
0,1709163872315,2443942101051,US,2019-10-03 05:26:33 UTC,$,0.0
1,1944180686907,2792791801915,DE,2020-01-11 23:37:09 UTC,€,0.0
2,3229976035488,4523443781792,DE,2021-02-12 07:17:45 UTC,€,0.0
3,2476680216736,3604864073888,US,2020-05-26 18:38:21 UTC,USD,0.0
4,4374091366560,3604864073888,DE,2021-12-06 07:28:39 UTC,EUR,0.0
5,2969055690912,3604864073888,GB,2020-12-15 22:34:11 UTC,£,0.0
6,2921555427488,3604864073888,GB,2020-11-30 22:04:27 UTC,GBP,0.0
7,3893348696224,3604864073888,GB,2021-05-17 04:51:34 UTC,GBP,0.0
8,4137335226528,5154056568992,DE,2021-08-23 06:11:57 UTC,euro,NaN
9,4295672103072,3604864073888,US,2021-10-26 05:30:09 UTC,USD,NaN


In [5]:
#
# drop transactions that have a non-value in the 'amount' column
# 
df = df.dropna(subset=['amount'])
df = df[~(df['amount'] <= 0.0)]
df.head(10)

,id,customer_id,customer_country_code,processed_at,currency,amount
17,2488365056160,3650538176672,US,2020-06-03 16:21:37 UTC,USD,88.48
20,4537404129440,6041435603104,US,2022-03-13 11:59:17 UTC,USD,71.35
21,4248044732576,5653137981600,GB,2021-10-07 20:47:28 UTC,GBP,28.28
22,4480696647840,5979289288864,GB,2022-02-07 01:12:24 UTC,GBP,34.84
24,4370463555744,5838666825888,US,2021-12-04 19:56:19 UTC,$,45.11
25,4243717882016,5645980270752,GB,2021-10-06 04:36:22 UTC,£,45.57
26,1178107772987,1557697134651,DE,2019-06-20 03:02:38 UTC,EUR,17.45
27,1779972014139,2547345850427,US,2019-10-11 20:37:19 UTC,USD,42.12
28,1811946602555,2595736780859,GB,2019-10-20 09:54:20 UTC,£,25.89
29,2190900265019,3189478654011,US,2020-04-24 16:27:15 UTC,USD,23.49


In [6]:
#
# ensure the 'processed_at' column is a datetime object, then drop the time
#
df['processed_at'] = pd.to_datetime(df['processed_at']).dt.date

df.head(10)

,id,customer_id,customer_country_code,processed_at,currency,amount
17,2488365056160,3650538176672,US,2020-06-03,USD,88.48
20,4537404129440,6041435603104,US,2022-03-13,USD,71.35
21,4248044732576,5653137981600,GB,2021-10-07,GBP,28.28
22,4480696647840,5979289288864,GB,2022-02-07,GBP,34.84
24,4370463555744,5838666825888,US,2021-12-04,$,45.11
25,4243717882016,5645980270752,GB,2021-10-06,£,45.57
26,1178107772987,1557697134651,DE,2019-06-20,EUR,17.45
27,1779972014139,2547345850427,US,2019-10-11,USD,42.12
28,1811946602555,2595736780859,GB,2019-10-20,£,25.89
29,2190900265019,3189478654011,US,2020-04-24,USD,23.49


In [7]:
#
# normalize the 'currency' columnn to use ISO codes like 'USD' and 'GBP'
#
df['currency'] = df['currency'].replace(['$', 'usd', 'US dollars', 'dollars'], 'USD')
df['currency'] = df['currency'].replace(['£', 'gbp', 'british pound', 'pounds'], 'GBP')
df['currency'] = df['currency'].replace(['€', 'eur', 'euro'], 'EUR')

# list all the currencies left in the dataframe after normalization
print(df["currency"].unique())

df.head(10)

['USD' 'GBP' 'EUR']


,id,customer_id,customer_country_code,processed_at,currency,amount
17,2488365056160,3650538176672,US,2020-06-03,USD,88.48
20,4537404129440,6041435603104,US,2022-03-13,USD,71.35
21,4248044732576,5653137981600,GB,2021-10-07,GBP,28.28
22,4480696647840,5979289288864,GB,2022-02-07,GBP,34.84
24,4370463555744,5838666825888,US,2021-12-04,USD,45.11
25,4243717882016,5645980270752,GB,2021-10-06,GBP,45.57
26,1178107772987,1557697134651,DE,2019-06-20,EUR,17.45
27,1779972014139,2547345850427,US,2019-10-11,USD,42.12
28,1811946602555,2595736780859,GB,2019-10-20,GBP,25.89
29,2190900265019,3189478654011,US,2020-04-24,USD,23.49


In [8]:
#
# let's sort transactions by date and currency
#
df = df.sort_values(by=['processed_at', 'currency'])

df.head(10)

,id,customer_id,customer_country_code,processed_at,currency,amount
24861,780128321595,1137657708603,DE,2018-12-11,EUR,23.55
25219,780136775739,1137657708603,GB,2018-12-11,GBP,21.46
4050,779169660987,1137115234363,US,2018-12-11,USD,23.90
18483,779238637627,1136340435003,US,2018-12-11,USD,16.90
33560,780005277755,1137581490235,US,2018-12-11,USD,16.90
35462,780194611259,1137686577211,US,2018-12-11,USD,29.90
41014,778796007483,1136916037691,US,2018-12-11,USD,25.90
41663,780115771451,1137650237499,US,2018-12-11,USD,23.90
47324,779885346875,1137528537147,US,2018-12-11,USD,16.90
49035,780270633019,1137729568827,US,2018-12-11,USD,51.80


In [9]:
#
# sum up revenue per currency, per day so we know what exchange rates to fetch
#
daily_totals = pd.DataFrame(df.groupby(['processed_at', df['currency']], )['amount'].sum())
daily_totals.rename(columns = {'amount':'daily_totals'}, inplace = True)
display(daily_totals)

daily_totals
processed_at currency              
2018-12-11   EUR              23.55
             GBP              21.46
             USD             256.80
2018-12-12   EUR              29.23
             GBP              41.56
...                             ...
2022-03-22   GBP              33.73
             USD             458.20
2022-03-23   EUR             101.24
             GBP              97.35
             USD             176.35

[3446 rows x 1 columns]

In [10]:
daily_totals = daily_totals.reset_index()
daily_totals.head()

,processed_at,currency,daily_totals
0,2018-12-11,EUR,23.55
1,2018-12-11,GBP,21.46
2,2018-12-11,USD,256.80
3,2018-12-12,EUR,29.23
4,2018-12-12,GBP,41.56


In [11]:
#
# https://chat.openai.com/chat/50313b79-b0bb-450f-bd9e-888542088b43
#
# Define function to get inflation rate based on CPI
def get_inflation_rate(date):
    url = f'https://www.statbureau.org/calculate-inflation-price-jsonp?country=united-states&start={date}&end={date}'
    response = requests.get(url)
    data = response.content.decode('utf-8')
    inflation_rate = float(data.split(':')[2].split(',')[0])
    return inflation_rate

# Define function to get exchange rate for a currency on a date
def get_exchange_rate(date, currency):
    if currency == 'USD':
        exchange_rate = 1.0
    else:
        url = f'https://api.exchangerate-api.com/v4/latest/USD/{currency}/{date}'
        response = requests.get(url)
        data = response.json()
        exchange_rate = data['rates'][currency]
    return exchange_rate

# Define function to adjust amount for inflation based on CPI
def adjust_for_inflation(amount, date):
    inflation_rate = get_inflation_rate(date)
    inflation_adjusted_amount = amount / inflation_rate
    return inflation_adjusted_amount

# Calculate exchange rate for each currency to USD on the date
daily_totals['exchange_rate'] = daily_totals.apply(lambda row: get_exchange_rate(row['processed_at'].strftime('%Y-%m-%d'), row['currency']), axis=1)

# Calculate USD amount for each transaction
# df['usd_amount'] = df['amount'] / df['exchange_rate']

# Adjust amount for inflation based on CPI
#df['inflation_adjusted_amount'] = df.apply(lambda row: adjust_for_inflation(row['usd_amount'], row['date'].strftime('%Y-%m-%d')), axis=1)

# Print the final DataFrame
#print(df)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
data = forex_converter.get_rate('EUR', 'USD', time.strptime('2021-03-03', '%Y-%m-%d').date())
print(data)

In [ ]:
#
# calculate exchange rates
#

forex_converter = CurrencyRates()

# if the currency is USD, the exchange_rate to USD is 1.0, otherwise fetch it using forex-python
def get_exchange_rate(processed_at, currency):
    if currency == 'USD':
        exchange_rate = 1.0
    else:
        exchange_rate = forex_converter.get_rate(currency, 'USD', time.strptime(processed_at, '%Y-%m-%d').date())
    return exchange_rate

daily_totals['exchange_rate'] = 1.0
#daily_totals['exchange_rate'] = daily_totals.apply(lambda row: get_exchange_rate(row['processed_at'].strftime('%Y-%m-%d'), row['currency']), axis=1)

daily_totals.info()